### Python Script to build the Knowledge Graph in Neo4j

#### 1. Install and import all the relevant packages

In [ ]:
!pip install neo4j

In [ ]:
import pandas as pd
from neo4j import GraphDatabase
from datetime import datetime

#### 2. connect to Neo4J

In [ ]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [ ]:
# Passe die Parameter an die erstellte Datenbank an
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="password")

#### 3. Import the datasets

# Report Dataset

In [ ]:
#Geben Sie den Pfadname der Datei Shortages_Report_Dataset an
shortages_report_path = r'C:\Users\jinji\Desktop\Data Science\Programming Projects\Neo4J für Teamprojekt\datensätze für neo4j\Shortages_Report_Dataset.xlsx'
shortages_report = pd.read_excel(shortages_report_path)
shortages_report.head()

# Substances Dataset

In [ ]:
#Geben Sie den Pfadname der Datei Substances_Dataset an
substances_data_path = r'C:\Users\jinji\Desktop\Data Science\Programming Projects\Neo4J für Teamprojekt\datensätze für neo4j\Substances_Dataset_entdoppelt.xlsx'
substances_data = pd.read_excel(substances_data_path)
substances_data.head()

# Drugs (medications) Dataset

In [ ]:
#Geben Sie den Pfadname der Datei Drugs_Dataset an
drugs_path = r'C:\Users\jinji\Desktop\Data Science\Programming Projects\Neo4J für Teamprojekt\datensätze für neo4j\Drugs_Dataset_14_01_24.xlsx'
drugs_data = pd.read_excel(drugs_path, dtype={'PZN': str})
drugs_data.head()

# Alternative Drugs Dataset

In [ ]:
#Geben Sie den Pfadname der Datei Drugs_Alternative_Dataset an
alternatives_data_path = r'C:\Users\jinji\Desktop\Data Science\Programming Projects\Neo4J für Teamprojekt\datensätze für neo4j\Alternative_PZN_keine_Doppelungen.xlsx'
alternatives_data = pd.read_excel(alternatives_data_path, dtype={'PZN': str, 'Alternative_PZN': str, 'ENR_Alternative': str })
alternatives_data.head()

# Check Data columns

In [ ]:
print(shortages_report.columns)

In [ ]:
print(drugs_data.columns)

# Constrainer

In [ ]:
####Constaints mit PZN Und ATC CODE
# Erstelle von Constaints, um sicher zu gehen, dass jeder Knoten nur einmal auftritt
# Constraint für 'drug' Label und 'PZN' Eigenschaft
constraint_drug_query = "CREATE CONSTRAINT drug_PZN_unique IF NOT EXISTS For (d:drug) Require d.PZN IS UNIQUE"
conn.query(constraint_drug_query)

# Constraint für 'substance' Label und 'atc_code' Eigenschaft
constraint_substance_query = "CREATE CONSTRAINT substance_atc_code_unique IF NOT EXISTS for (s:substance) require s.ATC_Code IS UNIQUE"
conn.query(constraint_substance_query)

constraint_drug_query = "CREATE CONSTRAINT substance_report_id_unique IF NOT EXISTS For (r:report) Require r.report_id IS UNIQUE"
conn.query(constraint_drug_query)

constraint_drug_query = "CREATE CONSTRAINT substance_producer_unique IF NOT EXISTS For (p:producer) Require p.producer IS UNIQUE"
conn.query(constraint_drug_query)

# Drug Nodes

In [ ]:
for index, row in drugs_data.iterrows():
    
    ## Collecting Properties
    PZN = str(row['PZN']).zfill(8)
    drug_name = row['Arzneimittlbezeichnung']
    Atc_Code = row ['Atc Code']
    license_holder = row['Zulassungsinhaber']
    dosage_form = row['Darreichungsform']
    treatment_group = row['Obergruppe']
    treatmen_group_ATC_Code = row ['Obergruppe ATC-Code']
    precise_upper_group = row['präzisere Obergruppe']
    paediatric_drug = row['Kinderarzneimittel']
    generic_drug = row ['Generikum']
    price = row['Preis']
    
    query = f"MERGE (n:drug {{name: '{drug_name}', PZN: '{PZN}', dosage_form: '{dosage_form}', license_holder: '{license_holder}', price: '{price}', alternative_to: '', precise_upper_group: '{precise_upper_group}', paediatric_drug: '{paediatric_drug}', generic_drug: '{generic_drug}', Atc_Code: '{Atc_Code}', treatment_group: '{treatment_group}', treatmen_group_ATC_Code: '{treatmen_group_ATC_Code}' }})"
    conn.query(query)

    print(f"Medikamenten-Knoten für PZN: {PZN} erstellt.")


# Alternative Drugs Nodes

In [ ]:
print(alternatives_data.columns)

In [ ]:
for index, row in alternatives_data.iterrows():
    name = row['Alternativpräparat']
    PZN = str(row['Alternative_PZN']).zfill(8)
    ENR = row['ENR_Alternative']
    alternative_to = str(row['PZN']).zfill(8)
    
    query = f"MERGE (d:drug {{drug_name: '{drug_name}', PZN: '{PZN}', ENR: '{ENR}', alternative_to: '{alternative_to}'}})"
    conn.query(query)
    print(f"Knoten mit PZN {PZN} wurde erstellt")


# Substance Nodes

In [ ]:
print(substances_data.columns)

In [ ]:
 # festlegen der Properties

for index, row in substances_data.iterrows():
    atc_code = row['ATC-Code']  
    sub_name = row['Wirkstoffe']
    treatment_name = row['Obergruppe']
    treatment_atc_code = row['Obergruppe ATC']
    treatment_subgroup_name = row['Präzisere Obergruppe']
    treatment_subgroup_atc_code = row['Präzisere Obergruppe ATC']
    relevant_to_supply = row['Versorgungsrelevant']
    
    query = f"MERGE (n:substance {{atc_code: '{atc_code}', name: '{sub_name}', treatment_name: '{treatment_name}',  treatment_atc_code: '{treatment_atc_code}', treatment_subgroup_name: '{treatment_subgroup_name}', treatment_subgroup_atc_code: '{treatment_subgroup_atc_code}', relevant_to_supply: '{relevant_to_supply}'}})"
    conn.query(query)
    
    print(f"Wirkstoffknoten für ATC_Code {atc_code} erstellt.")

# Reasons Nodes

In [ ]:
for index, row in shortages_report.iterrows():
    reason = row['Grund']
    #reason_typ = row['Art des Grundes']
    #assumption_about_the_reason= row['Anm. zum Grund']

    query = f"Merge (r:reason{{reason:'{reason}' }})"
    
    conn.query(query)

    print(f"Knoten für Grund '{reason}' erstellt.")


# Report Nodes

In [ ]:
print(shortages_report.columns)

In [ ]:
for index, row in shortages_report.iterrows():
    report_id = row['Report_ID']
    PZN = row['PZN']
    ENR = row['ENR']
    report_typ = row['Meldungsart']
    #begin = row['Beginn']
    #end = row['Ende']
    time_span = row ['Dauer (Tage)']
    #last_report = row['Datum der letzten Meldung']
    #first_report = row['Datum der Erstmeldung']
    ATC_Code = row['Atc Code']
    drugs_name = row['Arzneimittlbezeichnung']  # Hier ist der Spaltenname unverändert
    producer = row['Zulassungsinhaber']
    information_for_specialist_circles = row['Info an Fachkreise']
    dosage_form = row['Darreichungsform']
    
    begin = datetime.combine(row['Beginn'], datetime.min.time())
    end = datetime.combine(row['Ende'], datetime.min.time())
    last_report = datetime.combine(row['Datum der letzten Meldung'], datetime.min.time())
    first_report = datetime.combine(row['Datum der Erstmeldung'], datetime.min.time())
    
    
    query = (
    f"MERGE (r:report "
    f"{{"
    f"report_id: '{report_id}', "
    f"PZN: '{PZN}', "
    f"ENR: '{ENR}', "
    f"report_typ: '{report_typ}', "
    f"begin: date('{begin.strftime('%Y-%m-%d')}'), "
    f"end: date('{end.strftime('%Y-%m-%d')}'), "
    f"last_report: date('{last_report.strftime('%Y-%m-%d')}'), "
    f"first_report: date('{first_report.strftime('%Y-%m-%d')}'), " 
    f"time_span: '{time_span}',"
    f"ATC_Code: '{ATC_Code}', "
    f"drugs_name: '{drugs_name}', "
    f"producer: '{producer}', "
    f"information_for_specialist_circles: '{information_for_specialist_circles}', "
    f"dosage_form: '{dosage_form}'"
    f"}})"
    )
    

    conn.query(query)

    print(f"Berichtsknoten für Meldungs ID {report_id} erstellt.")


# Producer Nodes

In [ ]:
for index, row in shortages_report.iterrows():
    producer = row['Zulassungsinhaber']
    phone = row['Telefon']
    mail = row['E-Mail']
    ENR = row['ENR']
    query = f"MERGE (r:producer {{producer:'{producer}', phone: '{phone}', mail: '{mail}', ENR: '{ENR}'}})"
    
    conn.query(query)

    print(f"Knoten für Zulassungsinhaber '{producer}' erstellt.")


# Treatment Nodes

In [ ]:
for index, row in drugs_data.iterrows():
    treatment = row['Obergruppe']
    treatment_atc_code = row ['Obergruppe ATC-Code']
    query = f"MERGE (o:treatment {{treatment: '{treatment}'}})"
    conn.query(query)
    

# Connection ATC and PZN (has_substance)

In [ ]:
#Wählen Sie die Datei ATC_PZN_Connection aus
pzn_atc_path = r'C:\Users\jinji\Desktop\Data Science\Programming Projects\Neo4J für Teamprojekt\datensätze für neo4j\has_substance_fertig.xlsx'
pzn_atc = pd.read_excel (pzn_atc_path, dtype={'PZN': str} )
pzn_atc.head()

In [ ]:
# Beispielhafte Erstellung einer Verbindung zwischen den Knoten "Drug" und "Substance"
for index, row in pzn_atc.iterrows():
    PZN = str(row['PZN']).zfill(8)  
    atc_code = row['Atc Code'] 
    
    # Cypher-Abfrage zum Erstellen der Beziehung
    query = f"MATCH (d:drug {{PZN: '{PZN}'}}), (s:substance {{atc_code: '{atc_code}'}}) Merge (d)-[:has_substance]->(s)"
    conn.query(query)
    
    print(f"Verbindung zwischen PZN: {PZN} und Atc-Code: {atc_code} erstellt.")
    
     

# Connection Report ID and PZN (report_has)

In [ ]:
#Geben Sie den Pfadname der Datei ReportID_PZN_Connection an
report_drug_path = r'C:\Users\jinji\Desktop\Data Science\Programming Projects\Neo4J für Teamprojekt\datensätze für neo4j\report_has_fertig.xlsx'
report_drug = pd.read_excel (report_drug_path, dtype={'PZN': str, 'Report_ID': str})
report_drug.head()

In [ ]:
for index, row in report_drug.iterrows():
    report_id = str(row['Report_ID'])
    PZN = str(row['PZN']).zfill(8)
    query = f"MATCH (r:report {{report_id: '{report_id}'}}), (d:drug {{PZN: '{PZN}'}}) MERGE (r)-[:report_has]->(d)"
    conn.query(query)

    
    print(f"Verbindung zwischen report id: {report_id} und PZN: {PZN} erstellt.")


# Connection Reason and PZN (reason_influences)

In [ ]:
#Geben Sie den Pfadname der Datei ReportID_PZN_Connection an
reason_influences_path = r'C:\Users\jinji\Desktop\Data Science\Programming Projects\Neo4J für Teamprojekt\datensätze für neo4j\reason_influences_fertig.xlsx'
reason_influences = pd.read_excel (reason_influences_path, dtype={'PZN': str})
reason_influences.head()

In [ ]:
for index, row in reason_influences.iterrows():
    PZN = str(row['PZN']).zfill(8)
    reason = row['Grund']
    
     #Cypher-Abfrage zum Erstellen der Beziehung
    query = f"MATCH (d:drug {{PZN: '{PZN}'}}), (r:reason {{reason: '{reason}'}}) Merge (r)-[:reason_influences]->(d)"
    conn.query (query)
    
    print(f"Verbindung zwischen PZN: {PZN} und Gründen: {reason} erstellt.")

# Connection Report ID and Reason (because)

In [ ]:
for index, row in shortages_report.iterrows():
    report_id = str(row['Report_ID'])
    reason = row['Grund']
    
     #Cypher-Abfrage zum Erstellen der Beziehung
    query = f"MATCH (w:report {{report_id: '{report_id}'}}), (r:reason {{reason: '{reason}'}}) Merge (w)-[:because]->(r)"
    conn.query (query)
    
    print(f"Verbindung zwischen Report_ID: {report_id} und Grund: {reason} erstellt.")

# Connection Producer and PZN (producer_of)

In [ ]:
print(drugs_data.columns)

In [ ]:
for index, row in drugs_data.iterrows():
    producer = row['Zulassungsinhaber']
    PZN = str(row['PZN']).zfill(8)
    
    query = f"MATCH (d:drug {{PZN:'{PZN}'}}), (p:producer {{producer:'{producer}'}}) MERGE (p)-[:producer_of]->(d)"
    conn.query(query)


# Connection ATZ and Treatment (used_for)

In [ ]:
print(substances_data.columns)

In [ ]:
for index, row in substances_data.iterrows():
    atc_code = row['ATC-Code']
    treatment = row['Obergruppe']
    
    #Verbindugn zwischen den 
    query = f"MATCH (s:substance {{atc_code:'{atc_code}'}}), (t:treatment {{treatment:'{treatment}'}}) Merge (s)-[:used_for]->(t)"
    conn.query(query)


# Connection PZN and alternative PZN (HAS_ALTERNATIVE)

In [ ]:
print(alternatives_data.columns)

In [ ]:
for index, row in alternatives_data.iterrows():
    PZN = str(row['PZN']).zfill(8)
    alternative_PZN = str(row['Alternative_PZN']).zfill(8)
    
    # Erstelle die Beziehung
    create_relation_query = f"MATCH (drug1:drug {{PZN: '{PZN}'}}), (drug2:drug {{PZN: '{alternative_PZN}'}}) Merge (drug1)-[:HAS_ALTERNATIVE]->(drug2)"
    conn.query(create_relation_query)
    
    print(f"Beziehung zwischen PZN: {PZN} und Alternative_PZN: {alternative_PZN} erstellt.")


# NOT NECESSARY FOR THE KNOWLEDGE GRAPH - Script to split up PZNs of one cell into individual rows

In [ ]:
# Lade die Excel-Datei in ein Pandas DataFrame
file_path = '/Users/olesuhrmann/Documents/Studium KIT/7. Semester/Teamprojekt/Datensätze_aktuell/Alternativ_Verbindung.xlsx'  # Passe den Dateipfad an
df = pd.read_excel(file_path)

# Erstelle ein neues DataFrame für die aufgeteilten Zeilen
new_rows = []

# Iteriere durch jede Zeile des ursprünglichen DataFrames
for _, row in df.iterrows():
    # Teile die PZNs auf, falls sie durch ein Trennzeichen getrennt sind
    pzn_list = str(row['Alternative_PZN']).split(',')  # Passe die Spaltenbezeichnung 'PZN' an

    # Iteriere durch die aufgeteilten PZNs und erstelle eine neue Zeile für jede
    for pzn in pzn_list:
        new_row = row.copy()
        new_row['Alternative_PZN'] = pzn.strip()  # Entferne Leerzeichen um die PZN
        new_rows.append(new_row)

# Erstelle ein neues DataFrame mit den aufgeteilten Zeilen
new_df = pd.DataFrame(new_rows)

# Speichere das neue DataFrame in eine Excel-Datei
output_file_path = 'Test213.xlsx'  # Passe den Dateipfad an
new_df.to_excel(output_file_path, index=False)

print("Neue Zeilen wurden erstellt und die Datei wurde gespeichert.")
